In [29]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4" # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')
# bm.set_platform('cpu')

In [30]:
s =bm.array([0.5,1,1.5])
phi = bm.array([[0.3313,0.8148,0.4364],[0.8835,0.3621,0.2182],[0.3313,0.4527,0.8729]])
b = phi.T @ s
w = phi.T @ phi
w[bm.diag_indices_from(w)] = 0
print(w)
print(b)

Array(value=Array([[0.        , 0.7397555 , 0.62646115],
                   [0.7397555 , 0.        , 0.82969487],
                   [0.62646115, 0.82969487, 0.        ]]),
      dtype=float32)
Array(value=Array([1.5460999, 1.44855  , 1.74575  ]), dtype=float32)


In [31]:
lamb = 0.1
class LIF(bp.NeuGroupNS):
    def __init__(self, size, V_rest=0., V_reset=0., V_th=1., R=100., tau=100., t_ref= 0.05, name=None):
        # 初始化父类
        super(LIF, self).__init__(size=size, name=name)

        # 初始化参数
        self.V_rest = V_rest
        self.V_reset = V_reset
        self.V_th = V_th
        self.R = R
        self.tau = tau
        self.t_ref = t_ref  # 不应期时长

        # 初始化变量
        self.V = bm.Variable(bm.zeros(self.num) + V_reset)
        self.input = bm.Variable(bm.zeros(self.num))
        self.t_last_spike = bm.Variable(bm.ones(self.num) * -1e7)  # 上一次脉冲发放时间
        self.refractory = bm.Variable(bm.zeros(self.num, dtype=bool))  # 是否处于不应期
        self.spike = bm.Variable(bm.zeros(self.num, dtype=bool))  # 脉冲发放状态
        # self.Iext  = bm.Variable(bm.zeros(self.num))

        # 使用指数欧拉方法进行积分
        self.integral = bp.odeint(f=self.derivative, method='exp_auto')

    # 定义膜电位关于时间变化的微分方程
    def derivative(self, V, t, R, Iext):
        dvdt = (-V + self.V_rest + R * Iext) / self.tau
        return dvdt
    
    def update(self):
        t = bp.share.load('t')
        dt = bp.share.load('dt')
        # 以数组的方式对神经元进行更新
        refractory = (t - self.t_last_spike) <= self.t_ref  # 判断神经元是否处于不应期
        V = self.integral(self.V, t, self.R, self.input, dt=dt)  # 根据时间步长更新膜电位
        V = bm.where(refractory, self.V, V)  # 若处于不应期，则返回原始膜电位self.V，否则返回更新后的膜电位V
        spike = V > self.V_th  # 将大于阈值的神经元标记为发放了脉冲
        self.spike.value = spike  # 更新神经元脉冲发放状态
        self.t_last_spike.value = bm.where(spike, t, self.t_last_spike)  # 更新最后一次脉冲发放时间
        self.V.value = bm.where(spike, self.V_reset, V)  # 将发放了脉冲的神经元膜电位置为V_reset，其余不变
        self.refractory.value = bm.logical_or(refractory, spike)  # 更新神经元是否处于不应期
        self.input[:] = 0.  # 重置外界输入

In [32]:
class Spiking_LCA(bp.DynamicalSystemNS):
    def __init__(self, num, w, b, lamb, V_rest=0., V_reset=0., V_th=1., R=10., tau=10., t_ref= 0.05, scale=1.0, method='exp_auto'):
        super(Spiking_LCA, self).__init__()
        
        # parameter setting
        self.V_rest = V_rest
        self.V_reset = V_reset
        self.V_th = V_th
        self.R = R
        self.tau = tau
        self.t_ref = t_ref  # 不应期时长
        self.lamb  = lamb
        
        # network size
        num_neuron = int(num * scale)

        pars = dict(V_rest=V_rest, V_th=V_th, V_reset=V_reset, tau = tau, R =R, t_ref = t_ref)
        self.N = LIF(num_neuron, **pars)
        self.average_current = bm.Variable(bm.zeros(num_neuron))          # 平均电流记录
        # synapses
        self.N2N = bp.synapses.Exponential(pre=self.N,
                                            post=self.N,
                                            conn= bp.connect.All2All(),
                                            g_max= w,
                                            tau=1.,
                                            output=bp.synouts.CUBA(),
                                            method=method,
                                            comp_method='dense')
        
    # def update(self,tdi):
    #     self.N2N(tdi)
    #     self.N.input += 3
    #     self.N(tdi)
        
    def update(self):
        t  = bp.share.load('t')
        dt = bp.share.load('dt')
        self.N2N()
        self.average_current.value = self.average_current * t/(t + 1) + (b - self.N2N.g)/(t+ 1)
        firing_rate =  bm.maximum(self.average_current - self.lamb, 0.)
        # self.N.input = bm.where(firing_rate > 1e-7, self.V_th / (self.R * (1 - bm.exp((self.t_ref - 1 / firing_rate) / self.R))), self.V_th/self.R)
        self.N.input   = bm.where(firing_rate > 1e-7, self.V_th / (self.R * (1 - bm.exp((self.t_ref - 1 / firing_rate) / self.R))), self.V_th/self.R)
        self.N()

In [18]:
neuron_group = LIF(1)
net = Spiking_LCA(w.shape[0],w,b,0.1)

print(net.N2N.g.shape)

# runner = bp.DSRunner(neuron_group,monitors=['V'], inputs=['input',1],dt=0.01)
# runner.run(10)
# plt.plot(runner.mon.ts, runner.mon['V'])

(3,)


In [12]:
rng = np.random.RandomState(28)
m, n = 7500, 10000
# m, n = 3000, 5024
# random design
A = rng.randn(m, n)  # random design
A_norm = np.linalg.norm(A,ord=2,axis = 0,keepdims =True)
A = A /A_norm
x0 = rng.rand(n)
x0[x0 < 0.9] = 0
b_np = np.dot(A, x0)
b = np.dot(A.T,b_np)
w = np.dot(A.T,A)
w[np.diag_indices_from(w)] = 0
w = bm.array(w)
b = bm.array(b)
l = 0.5  # regularization parameter

In [33]:
# net    = Spiking_LCA(1,0,1)
# net    = Spiking_LCA(2,bm.array([[0,1],[1,0]]),bm.array([2,2]))
l = 0.1
net    = Spiking_LCA(w.shape[0],w,b,l)
total_period = 1000
runner = bp.DSRunner(net,monitors=['N.V','N.spike','N2N.g'], dt = 0.01)
runner.run(total_period)
# plt.plot(runner.mon.ts, runner.mon['N.V'])

Predict 100000 steps: :   0%|          | 0/100000 [00:00<?, ?it/s]

Predict 100000 steps: : 100%|██████████| 100000/100000 [00:08<00:00, 12210.04it/s]


In [34]:
size_num = runner.mon['N.spike'].shape[0]
spike_calculate = runner.mon['N.spike']
sum_along_columns = np.sum(spike_calculate, axis=0)/total_period
print(sum_along_columns)

[0.678 0.003 1.217]
